In [1]:
import pandas as pd
import pandas_ta as ta
import importlib
import os
import sys
import yfinance as yf

module_path = os.path.abspath(os.path.join("..", "src"))

if module_path not in sys.path:
    sys.path.append(module_path)

Input needed:

1. prices.csv
-date, asset, Close, Expected Return (from AR model), other components needed to run strategies

2. strategy_map.csv
- asset, strategy (e.g., Strategy05), ideal_proportion

Config Example:

config = {
    "starting_cash": 1000000,        # Initial capital
    "buy_pct": 0.05,               # Each buy = 5% of max cap
    "cash_floor_pct": 0.10,        # Minimum 10% cash must be held
    "cash_ceiling_pct": 0.30,      # If cash > 30%, consider buying gold
    "fee": 0.001,                  # Transaction fee (0.1%)
    "tp_pct": 0.10,                # Take-profit threshold (10%) 
    "sl_pct": 0.05                 # Stop-loss threshold (5%)
}

Notes: the tp, sl pct gonna make "dynamic" - different for each asset.

In [2]:
# Create some sample data to try now
config = {
    "starting_cash": 1000000,  # Initial capital
    "buy_pct": 1,  
    "tp_pct": 0.10,  # Take-profit threshold (10%)
    "sl_pct": 0.05,  # Stop-loss threshold (5%)
}

classname_to_filename = {
    "BollingerBandsBreakout": "bollinger_bands_breakout",
    "LarryWilliamsPriceAction": "larry_williams_price_action",
    "MACDBollingerBandsMeanReversion": "macd_bollinger_bands_mean_reversion",
    "MeanReversion": "mean_reversion",
    "MichaelHarrisPriceAction": "michael_harris_price_action",
    "Momentum": "momentum",
    "RSIDivergence": "rsi_divergence",
    "Scalping": "scalping",
    "VolumeSpikeReversal": "volume_spike_reversal",
}

#CHANGE FILE HERE
strategy_map = pd.read_csv("../data/experiments/asset_strategies_2_months_with_tpsl.csv")
strategy_map.head()

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Max. Trade Duration,Avg. Trade Duration,Profit Factor,Expectancy [%],SQN,Kelly Criterion,Strategy,Asset,Weight,Commissions [$]
0,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.000000,58778.750476,58778.750476,0.000000,-9.778786,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,BollingerBandsBreakout,AES,0.058779,NaN
1,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,5.128205,55067.499180,58778.750476,-6.313934,-9.778786,-34.388955,8.138026,...,1 days 00:00:00,1 days 00:00:00,0.0,-6.509081,NaN,NaN,LarryWilliamsPriceAction,AES,0.058779,27.376993
2,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,0.000000,58778.750476,58778.750476,0.000000,9.442870,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,MACDBollingerBandsMeanReversion,AES,0.058779,NaN
3,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,2.564103,58549.668271,58778.750476,-0.389736,-9.778786,-2.491650,0.978759,...,0 days 00:00:00,0 days 00:00:00,0.0,-0.357466,NaN,NaN,MeanReversion,AES,0.058779,28.066472
4,2025-01-02 00:00:00,2025-02-28 00:00:00,57 days 00:00:00,7.692308,55705.398517,58861.914976,-5.228679,-9.778786,-29.319905,7.817601,...,4 days 00:00:00,4 days 00:00:00,0.0,-5.410194,NaN,NaN,MichaelHarrisPriceAction,AES,0.058779,27.388732


In [3]:
idx= strategy_map.groupby("Asset")["Sharpe Ratio"].idxmax()
best = strategy_map.loc[idx]
strategy_map_new = best[["Asset", "Weight", "Strategy"]]
strategy_map_new = strategy_map_new.set_index("Asset")
strategy_map_new

,Weight,Strategy
Asset,,
ADA-USD,0.016802,LarryWilliamsPriceAction
ADI,0.028764,RSIDivergence
AES,0.058779,RSIDivergence
AKAM,0.056386,MeanReversion
BLDR,0.042793,MichaelHarrisPriceAction
CARR,0.029253,Scalping
CBRE,0.041216,Scalping
CSGP,0.033277,MeanReversion
DECK,0.043717,LarryWilliamsPriceAction


First function retrieves data from yf 

In [4]:
def get_asset_df(strategy_map, start, end):
    df_list = []
    for asset_name in strategy_map.index:
        # Retrieve yf finance data
        df = yf.Ticker(asset_name).history(start=start, end=end, actions=False)

        # Align date formatting
        df = df.reset_index()
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce").dt.strftime("%Y-%m-%d")

        df["Asset"] = asset_name
        df_list.append(df.reset_index())

    df_all = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_all

In [5]:
asset_df = get_asset_df(strategy_map=strategy_map_new, start="2023-01-01", end="2025-04-01")
asset_df.head()

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   

                           Close      Volume  
Date       Asset                              
2023-01-01 ADA-USD      0.249771   113369989  
           ETH-USD   1200.964844  2399674550  
           LINK-USD     5.622443   109175362  
           LTC-USD     70.815659   344383885  
           SOL-USD      9.982173   194221164

Second function implements AR model to derive expected return 

In [6]:
def compute_returns(asset_df):
    asset_df["Return"] = asset_df.groupby("Asset")["Close"].pct_change()
    return asset_df


asset_df_with_returns = compute_returns(asset_df)

In [7]:
asset_df_with_returns.head(10)

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
           UNI-USD       0     0.000099     0.000100     0.000099   
           XRP-USD       0     0.339923     0.339923     0.336332   
2023-01-02 ADA-USD       1     0.249778     0.255767     0.247368   
           ETH-USD       1  1201.103271  1219.860596  1195.214966   
           LINK-USD      1     5.622763     5.737175     5.567815   

                           Close      Volume    Return  
Date       Asset                                        
2023-01-01 ADA-USD      0.249771   113369989       NaN  
           ETH-USD   1200.964844  2399674550       NaN  
           LINK-USD     5.622443   109175362       NaN  
           LTC-USD     70.815659   344383885       NaN  
           SOL-USD      9.982173   194221164       NaN  
           UNI-USD      0.000100           5       NaN  
           XRP-USD      0.338763   290828851       NaN  
2023-01-02 ADA-USD      0.253828   159328803  0.016243  
           ETH-USD   1214.656616  3765758498  0.011401  
           LINK-USD     5.687627   179768004  0.011593

In [8]:
def generate_expected_returns(data, ar_results):
    """ Calculate expected returns using fitted AR models """
    new_data = data.reset_index().merge(ar_results, on="Asset", how="left").set_index(["Date", "Asset"])
    new_data["Expected Return"] = new_data["const"] + new_data["Return"] * new_data["ar.L1"]
    return new_data

In [9]:
ar_model_info = pd.read_csv("../data/processed/ar_model_2months.csv")
ar_model_info

,Asset,Best Model,BIC,const,ar.L1,sigma2
0,AES,"(1, 0, 0)",-1778.074591,-0.000457,-0.037837,0.000435
1,AKAM,"(1, 0, 0)",-1836.249257,-0.000660,0.000063,0.000371
2,ODFL,"(1, 0, 0)",-1881.544869,-0.000455,-0.022480,0.000328
3,TSLA,"(1, 0, 0)",-1422.498760,0.001576,0.012920,0.001156
4,STZ,"(1, 0, 0)",-2059.430404,-0.000819,-0.037615,0.000201
5,INTC,"(1, 0, 0)",-1537.758276,-0.001218,0.031652,0.000842
6,DECK,"(1, 0, 0)",-1730.903523,0.000040,-0.033131,0.000496
7,BLDR,"(1, 0, 0)",-1716.992545,-0.000737,-0.031178,0.000515
8,CBRE,"(1, 0, 0)",-2071.179275,0.001259,0.072816,0.000195
9,MCHP,"(1, 0, 0)",-1776.869305,-0.000790,-0.044913,0.000437


In [10]:
# Apply AR model
asset_df_with_AR = generate_expected_returns(asset_df_with_returns, ar_model_info)
asset_df_with_AR = asset_df_with_AR.drop(['Best Model','BIC', 'const','ar.L1', 'sigma2'], axis=1)
asset_df_with_AR

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
...                    ...          ...          ...          ...   
2025-03-31 SOL-USD     820   124.644028   127.696899   123.079239   
           STZ         561   181.860001   185.350006   180.899994   
           TSLA        561   249.309998   260.559998   243.360001   
           UNI-USD     820     0.000166     0.000168     0.000163   
           XRP-USD     820     2.136142     2.137319     2.027242   

                           Close      Volume    Return  Expected Return  
Date       Asset                                                         
2023-01-01 ADA-USD      0.249771   113369989       NaN              NaN  
           ETH-USD   1200.964844  2399674550       NaN              NaN  
           LINK-USD     5.622443   109175362       NaN              NaN  
           LTC-USD     70.815659   344383885       NaN              NaN  
           SOL-USD      9.982173   194221164       NaN              NaN  
...                          ...         ...       ...              ...  
2025-03-31 SOL-USD    124.694130  3206702485  0.000413         0.001245  
           STZ        183.520004     1823000 -0.001197        -0.000773  
           TSLA       259.160004   134008900 -0.016657         0.001361  
           UNI-USD      0.000166           3  0.000000        -0.000361  
           XRP-USD      2.089897  4506662719 -0.021649         0.004082  

[18673 rows x 8 columns]

In [11]:
def calculate_technical_indicators(df):
    # Core indicators
    df["EMA_50"] = ta.ema(df["Close"], length=50)
    df["EMA_200"] = ta.ema(df["Close"], length=200)
    df["RSI"] = ta.rsi(df["Close"], length=14)
    df["ATR"] = ta.atr(df["High"], df["Low"], df["Close"], length=7)

    # Bollinger Bands of length 20
    bbands = ta.bbands(df["Close"], length=20)
    bbands = bbands.rename(
        columns={
            "BBU_20_2.0": "Upper_Band",
            "BBM_20_2.0": "Middle_Band",
            "BBL_20_2.0": "Lower_Band",
            "BBB_20_2.0": "Band_Width",
            "BBP_20_2.0": "Percent_B",
        }
    )

    # Bollinger Bands of length 200
    bbands200 = ta.bbands(df["Close"], length=200)
    bbands200 = bbands200.rename(
        columns={
            "BBU_200_2.0": "Upper_Band_200",
            "BBM_200_2.0": "Middle_Band_200",
            "BBL_200_2.0": "Lower_Band_200",
            "BBB_200_2.0": "Band_Width_200",
            "BBP_200_2.0": "Percent_B_200",
        }
    )

    # MACD
    macd = ta.macd(df["Close"])
    macd = macd.rename(
        columns={
            "MACD_12_26_9": "MACD",
            "MACDh_12_26_9": "Histogram",
            "MACDs_12_26_9": "Signal",
        }
    )

    # Miscellaneous
    df["SMA_20"] = ta.sma(df["Close"], length=20)
    df["STD_20"] = ta.stdev(df["Close"], length=20)
    df["SMA_Volume_10"] = ta.sma(df["Volume"], length=10)
    df["Momentum"] = ta.mom(df["Close"], length=10)

    df = df.join([bbands, bbands200, macd])

    return df

In [12]:
enriched_df = calculate_technical_indicators(asset_df_with_AR)
enriched_df.head()

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   

                           Close      Volume  Return  Expected Return  EMA_50  \
Date       Asset                                                                
2023-01-01 ADA-USD      0.249771   113369989     NaN              NaN     NaN   
           ETH-USD   1200.964844  2399674550     NaN              NaN     NaN   
           LINK-USD     5.622443   109175362     NaN              NaN     NaN   
           LTC-USD     70.815659   344383885     NaN              NaN     NaN   
           SOL-USD      9.982173   194221164     NaN              NaN     NaN   

                     EMA_200  ...  Band_Width  Percent_B  Lower_Band_200  \
Date       Asset              ...                                          
2023-01-01 ADA-USD       NaN  ...         NaN        NaN             NaN   
           ETH-USD       NaN  ...         NaN        NaN             NaN   
           LINK-USD      NaN  ...         NaN        NaN             NaN   
           LTC-USD       NaN  ...         NaN        NaN             NaN   
           SOL-USD       NaN  ...         NaN        NaN             NaN   

                     Middle_Band_200  Upper_Band_200  Band_Width_200  \
Date       Asset                                                       
2023-01-01 ADA-USD               NaN             NaN             NaN   
           ETH-USD               NaN             NaN             NaN   
           LINK-USD              NaN             NaN             NaN   
           LTC-USD               NaN             NaN             NaN   
           SOL-USD               NaN             NaN             NaN   

                     Percent_B_200  MACD  Histogram  Signal  
Date       Asset                                             
2023-01-01 ADA-USD             NaN   NaN        NaN     NaN  
           ETH-USD             NaN   NaN        NaN     NaN  
           LINK-USD            NaN   NaN        NaN     NaN  
           LTC-USD             NaN   NaN        NaN     NaN  
           SOL-USD             NaN   NaN        NaN     NaN  

[5 rows x 29 columns]

Third function to add signal from strategies 

In [13]:
def load_data_and_apply_strategies(df_all, strategy_map):
    df_list = []

    for asset_name in strategy_map.index:
        # try:
        # Get strategy module + class
        strat_class_name = strategy_map.loc[asset_name, "Strategy"]
        strat_file_name = classname_to_filename[strat_class_name]

        strategy_module = importlib.import_module(
            f"strategies.custom.{strat_file_name}"
        )
        strategy_class = getattr(strategy_module, strat_class_name)
        strategy_instance = strategy_class()

        # Get asset's DataFrame slice
        asset_df = df_all.xs(asset_name, level="Asset").copy()

        # Apply strategy
        asset_df = strategy_instance.generate_signals(asset_df)

        # Restore asset label if removed
        asset_df["Asset"] = asset_name
        df_list.append(asset_df.reset_index())

        # except Exception as e:
        #     print(f"Error processing {asset_name} ({strat_class_name}): {e}")
        #     continue

    # Combine and restore multi-index
    df_with_signals = pd.concat(df_list).set_index(["Date", "Asset"]).sort_index()
    return df_with_signals

In [14]:
df_with_signals = load_data_and_apply_strategies(enriched_df, strategy_map_new)
df_with_signals

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
...                    ...          ...          ...          ...   
2025-03-31 SOL-USD     820   124.644028   127.696899   123.079239   
           STZ         561   181.860001   185.350006   180.899994   
           TSLA        561   249.309998   260.559998   243.360001   
           UNI-USD     820     0.000166     0.000168     0.000163   
           XRP-USD     820     2.136142     2.137319     2.027242   

                           Close      Volume    Return  Expected Return  \
Date       Asset                                                          
2023-01-01 ADA-USD      0.249771   113369989       NaN              NaN   
           ETH-USD   1200.964844  2399674550       NaN              NaN   
           LINK-USD     5.622443   109175362       NaN              NaN   
           LTC-USD     70.815659   344383885       NaN              NaN   
           SOL-USD      9.982173   194221164       NaN              NaN   
...                          ...         ...       ...              ...   
2025-03-31 SOL-USD    124.694130  3206702485  0.000413         0.001245   
           STZ        183.520004     1823000 -0.001197        -0.000773   
           TSLA       259.160004   134008900 -0.016657         0.001361   
           UNI-USD      0.000166           3  0.000000        -0.000361   
           XRP-USD      2.089897  4506662719 -0.021649         0.004082   

                         EMA_50     EMA_200  ...  Upper_Band_200  \
Date       Asset                             ...                   
2023-01-01 ADA-USD          NaN         NaN  ...             NaN   
           ETH-USD          NaN         NaN  ...             NaN   
           LINK-USD         NaN         NaN  ...             NaN   
           LTC-USD          NaN         NaN  ...             NaN   
           SOL-USD          NaN         NaN  ...             NaN   
...                         ...         ...  ...             ...   
2025-03-31 SOL-USD   212.360444  213.901973  ...     1022.763160   
           STZ       211.229446  213.599665  ...     1022.960437   
           TSLA      213.109076  214.053002  ...     1023.715009   
           UNI-USD   204.751864  211.923123  ...     1023.715670   
           XRP-USD   196.804336  209.835230  ...     1023.715293   

                     Band_Width_200  Percent_B_200       MACD  Histogram  \
Date       Asset                                                           
2023-01-01 ADA-USD              NaN            NaN        NaN        NaN   
           ETH-USD              NaN            NaN        NaN        NaN   
           LINK-USD             NaN            NaN        NaN        NaN   
           LTC-USD              NaN            NaN        NaN        NaN   
           SOL-USD              NaN            NaN        NaN        NaN   
...                             ...            ...        ...        ...   
2025-03-31 SOL-USD       759.624074       0.445365 -30.315152 -19.102428   
           STZ           758.571716       0.481524 -27.114526 -12.721441   
           TSLA          753.489978       0.527460 -18.263955  -3.096697   
           UNI-USD       753.543208       0.367294 -31.795331 -13.302458   
           XRP-USD       753.512206       0.368580 -41.867794 -18.699937   

                        Signal  TotalSignal  level_0  EMASignal  AvgVolume10  
Date       Asset                                                              
2023-01-01 ADA-USD         NaN            0      NaN        NaN          NaN  
           ETH-USD         NaN      

In [15]:
#Compute Volatility 
df_with_signals["rolling_std"] = (
    df_with_signals.groupby("Asset")["Close"]
    .transform(lambda x: x.rolling(window=14, min_periods=5).std())
)
df_with_signals

index         Open         High          Low  \
Date       Asset                                                    
2023-01-01 ADA-USD       0     0.246414     0.249771     0.244245   
           ETH-USD       0  1196.713623  1203.475342  1192.885376   
           LINK-USD      0     5.568981     5.628169     5.517975   
           LTC-USD       0    70.002823    71.328911    69.234886   
           SOL-USD       0     9.961036    10.052801     9.721011   
...                    ...          ...          ...          ...   
2025-03-31 SOL-USD     820   124.644028   127.696899   123.079239   
           STZ         561   181.860001   185.350006   180.899994   
           TSLA        561   249.309998   260.559998   243.360001   
           UNI-USD     820     0.000166     0.000168     0.000163   
           XRP-USD     820     2.136142     2.137319     2.027242   

                           Close      Volume    Return  Expected Return  \
Date       Asset                                                          
2023-01-01 ADA-USD      0.249771   113369989       NaN              NaN   
           ETH-USD   1200.964844  2399674550       NaN              NaN   
           LINK-USD     5.622443   109175362       NaN              NaN   
           LTC-USD     70.815659   344383885       NaN              NaN   
           SOL-USD      9.982173   194221164       NaN              NaN   
...                          ...         ...       ...              ...   
2025-03-31 SOL-USD    124.694130  3206702485  0.000413         0.001245   
           STZ        183.520004     1823000 -0.001197        -0.000773   
           TSLA       259.160004   134008900 -0.016657         0.001361   
           UNI-USD      0.000166           3  0.000000        -0.000361   
           XRP-USD      2.089897  4506662719 -0.021649         0.004082   

                         EMA_50     EMA_200  ...  Band_Width_200  \
Date       Asset                             ...                   
2023-01-01 ADA-USD          NaN         NaN  ...             NaN   
           ETH-USD          NaN         NaN  ...             NaN   
           LINK-USD         NaN         NaN  ...             NaN   
           LTC-USD          NaN         NaN  ...             NaN   
           SOL-USD          NaN         NaN  ...             NaN   
...                         ...         ...  ...             ...   
2025-03-31 SOL-USD   212.360444  213.901973  ...      759.624074   
           STZ       211.229446  213.599665  ...      758.571716   
           TSLA      213.109076  214.053002  ...      753.489978   
           UNI-USD   204.751864  211.923123  ...      753.543208   
           XRP-USD   196.804336  209.835230  ...      753.512206   

                     Percent_B_200       MACD  Histogram     Signal  \
Date       Asset                                                      
2023-01-01 ADA-USD             NaN        NaN        NaN        NaN   
           ETH-USD             NaN        NaN        NaN        NaN   
           LINK-USD            NaN        NaN        NaN        NaN   
           LTC-USD             NaN        NaN        NaN        NaN   
           SOL-USD             NaN        NaN        NaN        NaN   
...                            ...        ...        ...        ...   
2025-03-31 SOL-USD        0.445365 -30.315152 -19.102428 -11.212724   
           STZ            0.481524 -27.114526 -12.721441 -14.393084   
           TSLA           0.527460 -18.263955  -3.096697 -15.167259   
           UNI-USD        0.367294 -31.795331 -13.302458 -18.492873   
           XRP-USD        0.368580 -41.867794 -18.699937 -23.167857   

                     TotalSignal  level_0  EMASignal  AvgVolume10  rolling_std  
Date       Asset                                                                
2023-01-01 ADA-USD             0      NaN        NaN          NaN          NaN  
           ETH-USD             0      NaN        NaN          NaN          NaN  
           LIN

In [16]:
#Select the timeframe to run the auto_trade function on
start_date = '2025-03-01'
end_date = '2025-03-31'  

valid_df_with_signals = df_with_signals.loc[
    (df_with_signals.index.get_level_values('Date') >= start_date) &
    (df_with_signals.index.get_level_values('Date') <= end_date)]

valid_df_with_signals

index         Open         High          Low  \
Date       Asset                                                    
2025-03-01 ADA-USD     790     0.633052     0.674651     0.626733   
           ETH-USD     790  2237.942139  2279.873047  2144.090088   
           LINK-USD    790    14.816746    15.121204    14.244670   
           LTC-USD     790   127.963272   130.267380   123.187874   
           SOL-USD     790   148.030212   150.787155   138.684677   
...                    ...          ...          ...          ...   
2025-03-31 SOL-USD     820   124.644028   127.696899   123.079239   
           STZ         561   181.860001   185.350006   180.899994   
           TSLA        561   249.309998   260.559998   243.360001   
           UNI-USD     820     0.000166     0.000168     0.000163   
           XRP-USD     820     2.136142     2.137319     2.027242   

                           Close       Volume    Return  Expected Return  \
Date       Asset                                                           
2025-03-01 ADA-USD      0.659265    585631597  0.041409         0.002711   
           ETH-USD   2216.643311  16135258239 -0.009501        -0.000109   
           LINK-USD    14.788806    315668142 -0.001882         0.000255   
           LTC-USD    124.269943    805254661 -0.028853         0.001880   
           SOL-USD    143.675705   3305410077 -0.029415         0.003730   
...                          ...          ...       ...              ...   
2025-03-31 SOL-USD    124.694130   3206702485  0.000413         0.001245   
           STZ        183.520004      1823000 -0.001197        -0.000773   
           TSLA       259.160004    134008900 -0.016657         0.001361   
           UNI-USD      0.000166            3  0.000000        -0.000361   
           XRP-USD      2.089897   4506662719 -0.021649         0.004082   

                         EMA_50     EMA_200  ...  Band_Width_200  \
Date       Asset                             ...                   
2025-03-01 ADA-USD   199.478286  230.823067  ...      824.193076   
           ETH-USD   278.582797  250.582473  ...      821.484594   
           LINK-USD  268.237934  248.236267  ...      824.472493   
           LTC-USD   262.592131  247.002771  ...      828.338325   
           SOL-USD   257.928742  245.974641  ...      825.542392   
...                         ...         ...  ...             ...   
2025-03-31 SOL-USD   212.360444  213.901973  ...      759.624074   
           STZ       211.229446  213.599665  ...      758.571716   
           TSLA      213.109076  214.053002  ...      753.489978   
           UNI-USD   204.751864  211.923123  ...      753.543208   
           XRP-USD   196.804336  209.835230  ...      753.512206   

                     Percent_B_200        MACD   Histogram     Signal  \
Date       Asset                                                        
2025-03-01 ADA-USD        0.379004  -51.993984  -24.256830 -27.737154   
           ETH-USD        1.460418  118.880092  117.293796   1.586295   
           LINK-USD       0.385927   75.754519   59.334579  16.419940   
           LTC-USD        0.439913   49.836916   26.733581  23.103335   
           SOL-USD        0.449007   30.511198    5.926290  24.584908   
...                            ...         ...         ...        ...   
2025-03-31 SOL-USD        0.445365  -30.315152  -19.102428 -11.212724   
           STZ            0.481524  -27.114526  -12.721441 -14.393084   
           TSLA           0.527460  -18.263955   -3.096697 -15.167259   
           UNI-USD        0.367294  -31.795331  -13.302458 -18.492873   
           XRP-USD        0.368580  -41.867794  -18.699937 -23.167857   

                     TotalSignal  level_0  EMASignal  AvgVolume10  rolling_std  
Date       Asset                                                                
2025-03-01 ADA-USD             0      NaN        NaN          NaN     0.064066  
           ETH-USD             0      NaN        NaN    

In [17]:
#Commission Fee for Non-crypto Asset based on WeBull
def calculate_fee(amount):
    return max(0.00025 * amount, 0.50)

In [18]:
def auto_trade(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check
            
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= config["tp_pct"] or profit_pct <= -config["sl_pct"]
                ): 
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= config["tp_pct"]
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net Realised P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet Realised P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value.csv'")

    trade_df.to_csv("../data/misc/trade_log.csv", index=False)
    print("Trade log saved to 'trade_log.csv'")

    return trade_df, final_value

In [19]:
auto_trade(valid_df_with_signals, strategy_map_new, config)

Index(['ADA-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'SOL-USD', 'UNI-USD',
       'XRP-USD', 'ADI', 'AES', 'AKAM', 'BLDR', 'CARR', 'CBRE', 'CSGP', 'DECK',
       'DXCM', 'GM', 'GS', 'INTC', 'LII', 'LRCX', 'MCHP', 'MCO', 'MOH', 'ODFL',
       'PCG', 'PHM', 'PLD', 'STZ', 'TSLA'],
      dtype='object', name='Asset')

Final Portfolio Value: 995269.05
Total Profit: -4730.95
Number of Trades: 18

Net Realised P&L by Asset (lowest to highest):
asset
MCHP      -37999.660448
ADI       -30287.194144
LII        -1791.564308
XRP-USD    -1139.690156
UNI-USD     -726.669832
MOH         2077.847414
Name: signed_value, dtype: float64

Asset that pulled down profit the most: MCHP ($-37999.66)
Daily portfolio value saved to 'daily_portfolio_value.csv'
Trade log saved to 'trade_log.csv'


(      asset        date      action       price         units  signed_value
 0       LII  2025-03-03     BUY_NEW  586.979980  5.195480e+01 -30496.424882
 1       LII  2025-03-06     BUY_ADD  583.900024  2.774973e-01   -162.030664
 2   UNI-USD  2025-03-10     BUY_NEW    0.000158  5.797783e+06   -916.049651
 3       ADI  2025-03-10     BUY_NEW  216.259995  1.330952e+02 -28783.165230
 4       ADI  2025-03-12     BUY_ADD  206.619995  5.814890e+00  -1201.472533
 5       LII  2025-03-12     BUY_ADD  569.640015  1.201160e+00   -684.229058
 6       ADI  2025-03-13     SELL_SL  203.800003  1.389101e+02  28309.874880
 7   UNI-USD  2025-03-19     SELL_TP    0.000174  5.797783e+06   1008.814207
 8      MCHP  2025-03-20     BUY_NEW   51.000000  7.148215e+02 -36455.894348
 9       LII  2025-03-21     BUY_ADD  569.549988  1.517517e-02     -8.643017
 10      MOH  2025-03-21     BUY_NEW  309.929993  1.067752e+02 -33092.831793
 11  XRP-USD  2025-03-26     BUY_NEW    2.349088  7.862596e+03 -18469.930384

In [20]:
#Experiment with dynamic TP/SL
def auto_trade_exp(df, strategy_map, config):
    if not isinstance(df.index, pd.MultiIndex):
        df = df.set_index(["date", "asset"])

    assets = df.index.get_level_values(1).unique()
    print(assets)
    dates = df.index.get_level_values(0).unique()

    # Initialize portfolio and cash
    portfolio = {asset: {"units": 0, "entry_price": None} for asset in assets}
    cash = config["starting_cash"]
    trade_log = []
    daily_values = []

    for date in dates:
        # Calculate total portfolio value at start of day
        total_value = cash + sum(
            portfolio[asset]["units"] * df.loc[(date, asset), "Close"]
            for asset in assets
            if (date, asset) in df.index
        )

        for asset in assets:
            if (date, asset) not in df.index:
                continue
            row = df.loc[(date, asset)]
            current_price = row["Close"]
            signal = row["TotalSignal"]

            # Get max allocation for this asset
            ideal_proportion = strategy_map.loc[asset, "Weight"]
            max_asset_value = ideal_proportion * total_value
            current_allocation = portfolio[asset]["units"] * current_price
            remaining_allocation = max_asset_value - current_allocation
            buy_chunk = config["buy_pct"] * max_asset_value
            buy_amount = min(buy_chunk, remaining_allocation)

            # ---------- SELL CONDITIONS ----------
            # TP/SL check

            #Dynamic TP/SL
            volatility = row.get("rolling_std", None)
            
            tp_dynamic = (volatility / current_price) *  1.5
            sl_dynamic = (volatility / current_price) *  1
            
            if portfolio[asset]["units"] > 0:
                entry_price = portfolio[asset]["entry_price"]
                profit_pct = (current_price - entry_price) / entry_price
                if (
                    profit_pct >= tp_dynamic or profit_pct <= -sl_dynamic
                ):  # IMPLEMENT DYNAMIC TP AND SL
                    if asset.endswith("-USD"):  # Crypto
                        sell_price = current_price * 0.99  # apply 1% spread
                        proceeds = portfolio[asset]["units"] * sell_price
                    else:
                        fee = calculate_fee(portfolio[asset]["units"] * current_price)
                        proceeds = portfolio[asset]["units"] * current_price - fee
                    cash += proceeds
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": (
                                "SELL_TP"
                                if profit_pct >= tp_dynamic
                                else "SELL_SL"
                            ),
                            "price": current_price,
                            "units": portfolio[asset]["units"],
                        }
                    )
                    portfolio[asset] = {"units": 0, "entry_price": None}
                    continue  # skip further actions after selling

            # Strategy signal: SHORT → sell everything
            if signal == 1 and portfolio[asset]["units"] > 0:
                if asset.endswith("-USD"):  # Crypto
                    sell_price = current_price * 0.99  # apply 1% spread
                    proceeds = portfolio[asset]["units"] * sell_price
                else:
                    fee = calculate_fee(portfolio[asset]["units"] * current_price)
                    proceeds = portfolio[asset]["units"] * current_price - fee
                cash += proceeds
                trade_log.append(
                    {
                        "asset": asset,
                        "date": date,
                        "action": "SELL_SHORT",
                        "price": current_price,
                        "units": portfolio[asset]["units"],
                    }
                )
                portfolio[asset] = {"units": 0, "entry_price": None}
                continue

            # ---------- BUY CONDITIONS ----------
            if signal == 2:
                expected_return = row.get("Expected Return", None)
                baseline_fee_pct = 0.00025
                if expected_return is None or expected_return <= 2 * baseline_fee_pct:
                    continue  # don't buy if expected return is missing or too low

                # Check if enough cash remains after purchase
                if buy_amount > 0:
                    if asset.endswith("-USD"):  # Crypto
                        buy_price = current_price * 1.01  # apply 1% spread
                        new_units = buy_amount / buy_price
                    else:
                        fee = calculate_fee(buy_amount)
                        new_units = (buy_amount - fee) / current_price

                    old_units = portfolio[asset]["units"]
                    old_price = portfolio[asset]["entry_price"]

                    total_units = old_units + new_units
                    new_avg_price = (
                        (old_units * old_price + new_units * current_price)
                        / total_units
                        if old_units > 0
                        else current_price
                    )  # update price in portfolio so that we can calculate the profit/ loss accurately

                    # Update portfolio
                    portfolio[asset]["units"] = total_units
                    portfolio[asset]["entry_price"] = new_avg_price
                    cash -= buy_amount

                    # Log trade
                    trade_log.append(
                        {
                            "asset": asset,
                            "date": date,
                            "action": "BUY_ADD" if old_units > 0 else "BUY_NEW",
                            "price": current_price,
                            "units": new_units,
                        }
                    )
        eod_value = cash + sum(
            portfolio[asset]['units'] * df.loc[(date, asset), 'Close']
            for asset in assets if (date, asset) in df.index
        )
        # Build daily row with total value and each asset's value
        daily_row = {'date': date, 'portfolio_value': eod_value}
        for asset in assets:
            if (date, asset) in df.index:
                asset_price = df.loc[(date, asset), 'Close']
                asset_units = portfolio[asset]['units']
                daily_row[asset] = asset_units * asset_price
            else:
                daily_row[asset] = 0  # no data that day

        daily_values.append(daily_row)

    # ---------- FINAL SUMMARY ----------
    final_value = cash + sum(
        portfolio[asset]["units"] * df.loc[(dates[-1], asset), "Close"]
        for asset in assets
        if (dates[-1], asset) in df.index
    )

    total_profit = final_value - config["starting_cash"]

    print("\nFinal Portfolio Value:", round(final_value, 2))
    print("Total Profit:", round(total_profit, 2))
    print("Number of Trades:", len(trade_log))

    trade_df = pd.DataFrame(trade_log)

    if not trade_df.empty:
        # Add a signed P&L for each trade
        trade_df["signed_value"] = trade_df["price"] * trade_df["units"]
        trade_df["signed_value"] = trade_df.apply(
            lambda row: (
                -row["signed_value"]
                if row["action"].startswith("BUY")
                else row["signed_value"]
            ),
            axis=1,
        )

        # Net Realised P&L by asset
        pnl_per_asset = trade_df.groupby("asset")["signed_value"].sum().sort_values()

        print("\nNet Realised P&L by Asset (lowest to highest):")
        print(pnl_per_asset)

        worst_asset = pnl_per_asset.idxmin()
        worst_loss = pnl_per_asset.min()

        print(
            f"\nAsset that pulled down profit the most: {worst_asset} (${round(worst_loss, 2)})"
        )

    else:
        print("\n(No trades were executed during the period.)")
    
    pd.DataFrame(daily_values).to_csv("../data/misc/daily_portfolio_value_exp.csv", index=False)
    print("Daily portfolio value saved to 'daily_portfolio_value_exp.csv'")

    trade_df.to_csv("../data/misc/trade_log_exp.csv", index=False)
    print("Trade log saved to 'trade_log_exp.csv'")

    return trade_df, final_value

In [21]:
auto_trade_exp(valid_df_with_signals, strategy_map_new, config)

Index(['ADA-USD', 'ETH-USD', 'LINK-USD', 'LTC-USD', 'SOL-USD', 'UNI-USD',
       'XRP-USD', 'ADI', 'AES', 'AKAM', 'BLDR', 'CARR', 'CBRE', 'CSGP', 'DECK',
       'DXCM', 'GM', 'GS', 'INTC', 'LII', 'LRCX', 'MCHP', 'MCO', 'MOH', 'ODFL',
       'PCG', 'PHM', 'PLD', 'STZ', 'TSLA'],
      dtype='object', name='Asset')

Final Portfolio Value: 993596.13
Total Profit: -6403.87
Number of Trades: 21

Net Realised P&L by Asset (lowest to highest):
asset
ADI       -30252.936384
LII        -3000.649547
MCHP       -1642.018720
XRP-USD    -1138.263389
UNI-USD     -813.617414
MOH         1440.727880
Name: signed_value, dtype: float64

Asset that pulled down profit the most: ADI ($-30252.94)
Daily portfolio value saved to 'daily_portfolio_value_exp.csv'
Trade log saved to 'trade_log_exp.csv'


(      asset        date   action       price         units  signed_value
 0       LII  2025-03-03  BUY_NEW  586.979980  5.195480e+01 -30496.424882
 1       LII  2025-03-06  BUY_ADD  583.900024  2.774973e-01   -162.030664
 2   UNI-USD  2025-03-10  BUY_NEW    0.000158  5.797783e+06   -916.049651
 3       ADI  2025-03-10  BUY_NEW  216.259995  1.330952e+02 -28783.165230
 4       ADI  2025-03-12  BUY_ADD  206.619995  5.814890e+00  -1201.472533
 5       LII  2025-03-12  BUY_ADD  569.640015  1.201160e+00   -684.229058
 6       ADI  2025-03-13  SELL_SL  203.800003  1.389101e+02  28309.874880
 7       LII  2025-03-13  SELL_SL  558.510010  5.343345e+01  29843.118515
 8   UNI-USD  2025-03-15  SELL_TP    0.000170  5.797783e+06    985.623068
 9       LII  2025-03-18  BUY_NEW  581.369995  5.229066e+01 -30400.219819
 10      LII  2025-03-20  BUY_ADD  578.770020  2.396070e-01   -138.677339
 11     MCHP  2025-03-20  BUY_NEW   51.000000  7.139214e+02 -36409.992391
 12      LII  2025-03-21  BUY_ADD  569